In [51]:
import json
import numpy as np
import matplotlib.pyplot as plt
import torch

import nonlinear
import nonlinear_dce
from locally_connected import LocallyConnected
import torch.nn as nn

In [52]:
d = 4
s = 2
run_id = 0

with open(f"result_d{d}_seed{s}_modified", "r") as f:
    data = json.load(f)

print('-----------------------------\n' +
              f'| results with {d} Nodes {s} seed|\n' +
              '-----------------------------\n')

print("admg: ", data["admg"])

W1_thres = 0.3
W_truth = np.array(data["W_truth"])
W_truth[np.abs(W_truth) < W1_thres] = 0

# W_est_truth = np.array(data["W_est_truth"])
# W_est_truth[np.abs(W_est_truth) < W1_thres] = 0

# W_est_DAGMA_with_h = np.array(data["W_est_DAGMA_with_h"])
# W_est_DAGMA_with_h[np.abs(W_est_DAGMA_with_h) < W1_thres] = 0

# W_est_DAGMA_without_h = np.array(data["W_est_DAGMA_without_h"])
# W_est_DAGMA_without_h[np.abs(W_est_DAGMA_without_h) < W1_thres] = 0

W_est_best_random = np.array(data["best_random_run"]["W_est_random"])
W_est_best_random[np.abs(W_est_best_random) < W1_thres] = 0

for run in data["random_runs"]:
    if run["run_id"] == 0:
        W_id = np.array(run["W_est_random"])
        Sigma_id = np.array(run["Sigma_est_random"])
        loss_id = np.array(run["mle_loss_random_end"])
        h_id = np.array(run["h_val_random"])
        break

W_id[np.abs(W_id) < W1_thres] = 0

Sigma_truth = np.round(data["Sigma_truth"], 2)
# Sigma_est_truth = np.round(data["Sigma_est_truth"], 2)
# Sigma_est_DAGMA_with_h = np.round(data["Sigma_est_DAGMA_with_h"], 2)
# Sigma_est_DAGMA_without_h = np.round(data["Sigma_est_DAGMA_without_h"], 2)
Sigma_est_best_random = np.round(data["best_random_run"]["Sigma_est_random"], 2)
Sigma_id = np.round(Sigma_id, 2)

W2_thres = 0.2
Sigma_truth[np.abs(Sigma_truth) < W2_thres] = 0
Sigma_est_best_random[np.abs(Sigma_est_best_random) < W2_thres] = 0

h_val_best_random = np.array(data["best_random_run"]["h_val_random"])


print('\n' +
        f'|W|\n' +
        '\n')
print("W_truth: ", W_truth)
# print("W_est_truth: ", W_est_truth)
# print("W_est_DAGMA_with_h: ", W_est_DAGMA_with_h)
# print("W_est_DAGMA_without_h: ", W_est_DAGMA_without_h)
print("W_est_best_random: ", W_est_best_random)
print("W_id: ", W_id)

print('\n' +
        f'|Sigma|\n' +
        '\n')
print("Sigma_truth: ", Sigma_truth)
# print("Sigma_est_truth: ", Sigma_est_truth)
# print("Sigma_est_DAGMA_with_h: ", Sigma_est_DAGMA_with_h)
# print("Sigma_est_DAGMA_without_h: ", Sigma_est_DAGMA_without_h)
print("Sigma_est_best_random: ", Sigma_est_best_random)
print("Sigma_id: ", Sigma_id)

print('\n' +
        f'|mle_loss|\n' +
        '\n')
print("mle_loss_truth: ", data["mle_loss_truth"])
# print("mle_loss_est_truth: ", data["mle_loss_truth_end"])
# print("mle_loss_est_DAGMA_with_h: ", data["mle_loss_DAGMA_with_h"])
# print("mle_loss_est_DAGMA_without_h: ", data["mle_loss_DAGMA_without_h"])
# print("mle_loss_est_best_random: ", data["best_random_run"]["mle_loss_random_end"])
print("mle_loss_id: ", loss_id)

print('\n' +
        f'|h_val|\n' +
        '\n')
print("h_val_best_random: ", h_val_best_random)
print("h_val_id: ", h_id)


-----------------------------
| results with 4 Nodes 2 seed|
-----------------------------

admg:  {'0': {'parents': [], 'spouses': []}, '1': {'parents': [0], 'spouses': []}, '2': {'parents': [1], 'spouses': [3]}, '3': {'parents': [], 'spouses': [2]}}

|W|


W_truth:  [[0.         3.97345821 0.         0.        ]
 [0.         0.         1.99255812 0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]
W_est_best_random:  [[0.         3.76557063 0.         0.        ]
 [0.         0.         1.87218355 0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]
W_id:  [[0.         3.76557063 0.         0.        ]
 [0.         0.         1.87218355 0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]]

|Sigma|


Sigma_truth:  [[ 1.    0.    0.    0.  ]
 [ 0.    1.    0.    0.  ]
 [ 0.    0.    1.   -0.57]
 [ 0.    0.   -0.57  1.  ]]
Sig

In [53]:
def f1_from_nonzero(A_true, A_est):
    A_true_nz = (A_true != 0)
    A_est_nz  = (A_est  != 0)

    TP = np.sum(A_true_nz & A_est_nz)
    FP = np.sum(~A_true_nz & A_est_nz)
    FN = np.sum(A_true_nz & ~A_est_nz)

    precision = TP / (TP + FP + 1e-12)
    recall    = TP / (TP + FN + 1e-12)
    f1        = 2 * precision * recall / (precision + recall + 1e-12)

    return TP, FP, FN, precision, recall, f1


In [54]:
TP_W1, FP_W1, FN_W1, precision_W1, recall_W1, f1_W1 = f1_from_nonzero(W_truth, W_est_best_random)
print("TP_W1: ", TP_W1)
print("FP_W1: ", FP_W1)
print("FN_W1: ", FN_W1)
print("precision_W1: ", precision_W1)
print("Trecall_W1: ", recall_W1)
print("f1_W1: ", f1_W1)

TP_W1:  2
FP_W1:  0
FN_W1:  0
precision_W1:  0.9999999999995
Trecall_W1:  0.9999999999995
f1_W1:  0.9999999999989999


In [55]:
TP_W2, FP_W2, FN_W2, precision_W2, recall_W2, f1_W2 = f1_from_nonzero(np.tril(Sigma_truth, k=-1), np.tril(Sigma_est_best_random, k=-1))
print("TP_W2: ", TP_W2)
print("FP_W2: ", FP_W2)
print("FN_W2: ", FN_W2)
print("precision_W2: ", precision_W2)
print("Trecall_W2: ", recall_W2)
print("f1_W2: ", f1_W2)

TP_W2:  1
FP_W2:  0
FN_W2:  0
precision_W2:  0.9999999999989999
Trecall_W2:  0.9999999999989999
f1_W2:  0.9999999999984999
